Imports and Connection to SSMS database

In [1]:
import warnings
warnings.simplefilter('ignore')

import pyodbc
import pandas as pd
import sqlite3

server = 'LAPTOP-8R8HAJSC\\SQLEXPRESS01'
source_db = 'United Outdoors Source'
northwind_db = 'NorthWind'

source_connectionString = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={source_db};Trusted_Connection=yes;'
northwind_connectionString = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={northwind_db};Trusted_Connection=yes;'

source_connection = pyodbc.connect(source_connectionString)
northwind_connection = pyodbc.connect(northwind_connectionString)

Deleting all tables in database

In [2]:
cursor = source_connection.cursor()
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_type = 'BASE TABLE'")

tables = cursor.fetchall()

# Drop foreign key constraints
cursor.execute("""
    SELECT 
        fk.name AS FK_name,
        tp.name AS table_name
    FROM 
        sys.foreign_keys AS fk
        INNER JOIN sys.tables AS tp ON fk.parent_object_id = tp.object_id
""")
foreign_keys = cursor.fetchall()

for fk in foreign_keys:
    fk_name = fk.FK_name
    table_name = fk.table_name
    try:
        cursor.execute(f"ALTER TABLE {table_name} DROP CONSTRAINT {fk_name}")
    except Exception as e:
        print(f"Error dropping foreign key {fk_name} on {table_name}: {e}")

# Drop each table
for table in tables:
    table_name = table[0]
    try:
        cursor.execute(f"DROP TABLE {table_name}")
    except Exception as e:
        print(f"Error dropping table {table_name}: {e}")

# Commit the changes
source_connection.commit()

Creating all tables in database

In [3]:
import pyodbc

server = 'LAPTOP-8R8HAJSC\\SQLEXPRESS01'
source_db = 'United Outdoors Source'

source_connectionString = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={source_db};Trusted_Connection=yes;'

source_connection = pyodbc.connect(source_connectionString)
cursor = source_connection.cursor()

create_tables = [
    """
    CREATE TABLE Department (
        dept_id INT PRIMARY KEY,
        dept_name VARCHAR(100),
        dept_head_id INT
    );
    """,
    """
    CREATE TABLE Bonus (
        emp_id INT,
        bonus_date DATE,
        bonus_amount DECIMAL(10, 2),
        PRIMARY KEY (emp_id, bonus_date, bonus_amount)
    );
    """,
    """
    CREATE TABLE State (
        state_id VARCHAR (100) PRIMARY KEY,
        state_name VARCHAR(100),
        state_capital VARCHAR(100),
        country VARCHAR(100),
        region VARCHAR(100)
    );
    """,
    """
    CREATE TABLE Employee (
        emp_id INT PRIMARY KEY,
        manager_id INT,
        emp_fname VARCHAR(100),
        emp_lname VARCHAR(100),
        dept_id INT FOREIGN KEY REFERENCES Department(dept_id),
        city VARCHAR(100),
        state_fk VARCHAR(100) FOREIGN KEY REFERENCES State(state_id),
        ssn VARCHAR(11),
        salary DECIMAL(10, 2),
        start_date DATE,
        birth_date DATE,
        bene_health_ins BIT,
        bene_life_ins BIT,
        bene_day_care BIT,
        sex CHAR(1)
    );
    """,
    """
    CREATE TABLE Shippers (
        ShipperID INT PRIMARY KEY,
        CompanyName VARCHAR(100),
        Phone VARCHAR(20)
    );
    """,
    """
    CREATE TABLE Customer (
        CustomerID VARCHAR(100) PRIMARY KEY,
        CompanyName VARCHAR(100),
        ContactName VARCHAR(100),
        ContactTitle VARCHAR(100),
        Address VARCHAR(100),
        City VARCHAR(100),
        Region VARCHAR(100),
        PostalCode VARCHAR(100),
        Country VARCHAR(100),
        Phone VARCHAR(100),
        Fax VARCHAR(100),
        State VARCHAR(100) FOREIGN KEY REFERENCES State(state_id)
    );
    """,
    """
    CREATE TABLE Orders (
        OrderID INT PRIMARY KEY,
        CustomerID VARCHAR(100) FOREIGN KEY REFERENCES Customer(customerID),
        EmployeeID INT FOREIGN KEY REFERENCES Employee(emp_id),
        OrderDate VARCHAR(100),
        RequiredDate VARCHAR(100),
        ShippedDate VARCHAR(100),
        ShipVia INT,
        Freight DECIMAL(10,2),
        ShipName VARCHAR(100),
        ShipAddress VARCHAR(255),
        ShipCity VARCHAR(100),
        ShipRegion VARCHAR(100),
        ShipPostalCode VARCHAR(10),
        ShipCountry VARCHAR(100)
    );
    """,
    """
    CREATE TABLE Order_Details (
        OrderID INT FOREIGN KEY REFERENCES Orders(OrderID),
        ProductID INT,
        UnitPrice DECIMAL(10,2),
        Quantity INT,
        Discount FLOAT,
        PRIMARY KEY (OrderID, ProductID)
    );
    """,
    """
    CREATE TABLE Sales_Order (
        id INT PRIMARY KEY,
        cust_id VARCHAR(100) FOREIGN KEY REFERENCES Customer(customerID),
        order_date DATE,
        region VARCHAR(100),
        sales_rep VARCHAR(100)
    );
    """,
    """
    CREATE TABLE Sales_Order_Item (
        id INT PRIMARY KEY,
        sales_id INT FOREIGN KEY REFERENCES Sales_Order(id),
        prod_id INT,
        quantity INT,
        ship_date DATE
    );
    """,
    """
    CREATE TABLE Categories (
        CategoryID INT PRIMARY KEY,
        CategoryName VARCHAR(100),
        Description TEXT,
        Picture VARBINARY(MAX)
    );
    """,
    """
    CREATE TABLE Production_ProductCategory (
        ProductCategoryID INT PRIMARY KEY,
        Name VARCHAR(100),
        rowguid UNIQUEIDENTIFIER,
        ModifiedDate DATE
    );
    """,
    """
    CREATE TABLE Products (
        ProductID INT PRIMARY KEY,
        ProductName VARCHAR(100),
        ProductNumber VARCHAR(50),
        MakeFlag BIT,
        FinishedGoodsFlag BIT,
        Color VARCHAR(50),
        SafetyStockLevel INT,
        ReorderLevel INT,
        StandardCost DECIMAL(10,2),
        ListPrice DECIMAL(10,2),
        Size VARCHAR(50),
        SizeUnitMeasureCode VARCHAR(10),
        Weight DECIMAL(10,2),
        WeightUnitMeasureCode VARCHAR(10),
        DaysToManufacture INT,
        ProductLine VARCHAR(50),
        Class CHAR(1),
        Style CHAR(1),
        ProductSubcategoryID INT,
        ProductModelID INT,
        SellStartDate DATE,
        SellEndDate DATE,
        DiscontinuedDate DATE,
        rowguid UNIQUEIDENTIFIER,
        ModifiedDate DATE,
        ProductionCategoryID INT
    );
    """,
    """
    CREATE TABLE Product (
        id INT PRIMARY KEY,
        name VARCHAR(100),
        description TEXT,
        prod_size VARCHAR(50),
        color VARCHAR(50),
        quantity INT,
        unit_price DECIMAL(10,2),
        product_name VARCHAR(100),
        category VARCHAR(100)
    );
    """,
    """
    CREATE TABLE Production_BillOfMaterials (
        BillOfMaterialsID INT PRIMARY KEY,
        ProductAssemblyID INT,
        ComponentID INT,
        StartDate DATE,
        EndDate DATE,
        UnitMeasureCode VARCHAR(10),
        BOMLevel INT,
        PerAssemblyQTY DECIMAL(10,2),
        ModifiedDate DATE
    );
    """,
    """
    CREATE TABLE Sales_SalesTerritory (
        TerritoryID INT PRIMARY KEY,
        Name VARCHAR(50),
        CountryRegionCode VARCHAR(10),
        [Group] VARCHAR(50),
        SalesYTD DECIMAL(10,2),
        SalesLastYear DECIMAL(10,2),
        CostYTD DECIMAL(10,2),
        CostLastYear DECIMAL(10,2),
        rowguid UNIQUEIDENTIFIER
    );
    """,
    """
    CREATE TABLE Sales_Customer (
        CustomerID INT PRIMARY KEY,
        PersonID INT,
        StoreID INT,
        TerritoryID INT,
        AccountNumber VARCHAR(25),
        rowguid UNIQUEIDENTIFIER,
        ModifiedDate DATE
    );
    """,
    """
    CREATE TABLE Sales_Store (
        BusinessEntityID INT PRIMARY KEY,
        Name VARCHAR(100),
        SalesPersonID INT,
        rowguid UNIQUEIDENTIFIER,
        ModifiedDate DATE
    );
    """,
    """
    CREATE TABLE Sales_SalesOrderHeader (
        SalesOrderID INT PRIMARY KEY,
        RevisionNumber INT,
        OrderDate DATE,
        DueDate DATE,
        ShipDate DATE,
        Status INT,
        OnlineOrderFlag BIT,
        SalesOrderNumber VARCHAR(25),
        PurchaseOrderNumber VARCHAR(25),
        AccountNumber VARCHAR(25),
        CustomerID INT FOREIGN KEY REFERENCES Sales_Customer(CustomerID),
        SalesPersonID INT,
        TerritoryID INT FOREIGN KEY REFERENCES Sales_SalesTerritory(TerritoryID),
        BillToAddressID INT,
        ShipToAddressID INT,
        ShipMethodID INT,
        CreditCardID INT,
        CreditCardApprovalCode VARCHAR(15),
        CurrencyRateID INT,
        SubTotal DECIMAL(10,2),
        TaxAmt DECIMAL(10,2),
        Freight DECIMAL(10,2),
        TotalDue DECIMAL(10,2),
        Comment TEXT,
        rowguid UNIQUEIDENTIFIER,
        ModifiedDate DATE
    );
    """,
    """
    CREATE TABLE Sales_SalesOrderDetail (
        SalesOrderID INT FOREIGN KEY REFERENCES Sales_SalesOrderHeader(SalesOrderID),
        SalesOrderDetailID INT PRIMARY KEY,
        CarrierTrackingNumber VARCHAR(50),
        OrderQty INT,
        ProductID INT FOREIGN KEY REFERENCES Products(ProductID),
        SpecialOfferID INT,
        UnitPrice DECIMAL(10,2),
        UnitPriceDiscount DECIMAL(10,2),
        LineTotal DECIMAL(10,2),
        rowguid UNIQUEIDENTIFIER,
        ModifiedDate DATE
    );
    """,
    """
    CREATE TABLE HumanResources_Department (
        DepartmentID INT PRIMARY KEY,
        Name VARCHAR(100),
        GroupName VARCHAR(100),
        ModifiedDate DATE
    );
    """,
    """
    CREATE TABLE HumanResources_Employee (
        BusinessEntityID INT PRIMARY KEY,
        NationalIDNumber VARCHAR(20),
        LoginID VARCHAR(50),
        OrganizationNode VARCHAR(100),
        OrganizationLevel SMALLINT,
        JobTitle VARCHAR(50),
        BirthDate DATE,
        MaritalStatus CHAR(1),
        Gender CHAR(1),
        HireDate DATE,
        SalariedFlag BIT,
        VacationHours SMALLINT,
        SickLeaveHours SMALLINT,
        CurrentFlag BIT,
        rowguid UNIQUEIDENTIFIER,
        ModifiedDate DATE,
        DepartmentID INT FOREIGN KEY REFERENCES HumanResources_Department(DepartmentID)
    );
    """,
    """
    CREATE TABLE Person_Person (
        BusinessEntityID INT PRIMARY KEY,
        PersonType CHAR(2),
        NameStyle BIT,
        Title VARCHAR(10),
        FirstName VARCHAR(50),
        MiddleName VARCHAR(50),
        LastName VARCHAR(50),
        Suffix VARCHAR(10),
        EmailPromotion INT,
        rowguid UNIQUEIDENTIFIER,
        ModifiedDate DATE
    );
    """,
    """
    CREATE TABLE Person_Address (
        AddressID INT PRIMARY KEY,
        AddressLine1 VARCHAR(100),
        AddressLine2 VARCHAR(100),
        City VARCHAR(50),
        StateProvince VARCHAR(50),
        PostalCode VARCHAR(15),
        SpatialLocation VARCHAR(255),
        rowguid UNIQUEIDENTIFIER,
        ModifiedDate DATE,
        BusinessEntityID INT FOREIGN KEY REFERENCES Person_Person(BusinessEntityID)
    );
    """,
    """
    CREATE TABLE Purchasing_Vendor (
        BusinessEntityID INT PRIMARY KEY,
        AccountNumber VARCHAR(25),
        Name VARCHAR(100),
        CreditRating INT,
        PreferredVendorStatus BIT,
        ActiveFlag BIT,
        PurchasingWebServiceURL VARCHAR(255),
        ModifiedDate DATE
    );
    """,
    """
    CREATE TABLE Purchasing_PurchaseOrderHeader (
        PurchaseOrderID INT PRIMARY KEY,
        RevisionNumber INT,
        Status INT,
        EmployeeID INT,
        VendorID INT FOREIGN KEY REFERENCES Purchasing_Vendor(BusinessEntityID),
        ShipMethodID INT,
        OrderDate DATE,
        ShipDate DATE,
        SubTotal DECIMAL(10,2),
        TaxAmt DECIMAL(10,2),
        Freight DECIMAL(10,2),
        TotalDue DECIMAL(10,2),
        ModifiedDate DATE
    );
    """,
    """
    CREATE TABLE Purchasing_PurchaseOrderDetail (
        PurchaseOrderID INT FOREIGN KEY REFERENCES Purchasing_PurchaseOrderHeader(PurchaseOrderID),
        PurchaseOrderDetailID INT PRIMARY KEY,
        DueDate DATE,
        OrderQty INT,
        ProductID INT FOREIGN KEY REFERENCES Products(ProductID),
        UnitPrice DECIMAL(10,2),
        LineTotal DECIMAL(10,2),
        ReceivedQty DECIMAL(10,2),
        RejectedQty DECIMAL(10,2),
        StockedQty DECIMAL(10,2),
        ModifiedDate DATE
    );
    """,
    """
    CREATE TABLE Territory (
        TerritoryID INT PRIMARY KEY,
        Name VARCHAR(100),
        CountryRegionCode VARCHAR(10),
        RegionDescription VARCHAR(100)
    );
    """,
    """
    CREATE TABLE EmployeeTerritories (
        EmployeeID INT FOREIGN KEY REFERENCES Employee(emp_id),
        TerritoryID INT FOREIGN KEY REFERENCES Territory(TerritoryID),
        PRIMARY KEY (EmployeeID, TerritoryID)
    );
    """
]
for table_query in create_tables:
    try:
        cursor.execute(table_query)
        print("Table created successfully.")
    except Exception as e:
        print(f"Error creating table: {e}")

source_connection.commit()

Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.


Creating Dataframes from reading data from files

In [4]:
#Dataframes from aenc.sqlite
aenc_connection = sqlite3.connect('Data\\aenc.sqlite')
sqlite_cursor = aenc_connection.cursor()

bonus_df = pd.read_sql_query('SELECT * FROM Bonus', aenc_connection)
customer_df = pd.read_sql_query('SELECT * FROM Customer', aenc_connection)
department_df = pd.read_sql_query('SELECT * FROM Department', aenc_connection)
employee_df = pd.read_sql_query('SELECT * FROM Employee', aenc_connection)
product_df = pd.read_sql_query('SELECT * FROM Product', aenc_connection)
sales_order_df = pd.read_sql_query('SELECT * FROM Sales_Order', aenc_connection)
sales_order_item_df = pd.read_sql_query('SELECT * FROM Sales_Order_Item', aenc_connection)
state_df = pd.read_sql_query('SELECT * FROM State', aenc_connection)

#Dataframes from NorthWind.txt
sqlite_cursor = northwind_connection.cursor()

categories_df = pd.read_sql_query('SELECT * FROM Categories', northwind_connection)
customer_customer_demo_df = pd.read_sql_query('SELECT * FROM CustomerCustomerDemo', northwind_connection)
customer_demographics_df = pd.read_sql_query('SELECT * FROM CustomerDemographics', northwind_connection)
customers_df = pd.read_sql_query('SELECT * FROM Customers', northwind_connection)
employees_df = pd.read_sql_query('SELECT * FROM Employees', northwind_connection)
employee_territories_df = pd.read_sql_query('SELECT * FROM EmployeeTerritories', northwind_connection)
order_details_df = pd.read_sql_query('SELECT * FROM "Order Details"', northwind_connection)
orders_df = pd.read_sql_query('SELECT * FROM Orders', northwind_connection)
products_df = pd.read_sql_query('SELECT * FROM Products', northwind_connection)
region_df = pd.read_sql_query('SELECT * FROM Region', northwind_connection)
shippers_df = pd.read_sql_query('SELECT * FROM Shippers', northwind_connection)
suppliers_df = pd.read_sql_query('SELECT * FROM Suppliers', northwind_connection)
territories_df = pd.read_sql_query('SELECT * FROM Territories', northwind_connection)

#Dataframes from AdventureWorks.ldb

#hier

Merge Customer (and convert the keys to string)

In [5]:
customers_df['CustomerID'] = customers_df['CustomerID'].astype(str)
customer_df['id'] = customer_df['id'].astype(str)

customer_merge_df = pd.merge(customers_df, customer_df, left_on= 'CustomerID', how= 'outer', right_on= 'id', indicator= 'true')

customer_merge_df.drop(columns=['id'], inplace=True)
customer_merge_df['Address'] = customer_merge_df['Address'].combine_first(customer_merge_df['address'])
customer_merge_df['City'] = customer_merge_df['City'].combine_first(customer_merge_df['city'])
customer_merge_df['Phone'] = customer_merge_df['Phone'].combine_first(customer_merge_df['phone'])
customer_merge_df['CompanyName'] = customer_merge_df['CompanyName'].combine_first(customer_merge_df['company_name'])
customer_merge_df['PostalCode'] = customer_merge_df['zip'].combine_first(customer_merge_df['PostalCode'])
customer_merge_df['Region'] = customer_merge_df['Region'].fillna('NULL')
customer_merge_df.drop(columns=['address', 'city', 'phone', 'company_name', 'zip', 'true', 'fname', 'lname'], inplace=True)

print(customer_merge_df.columns.tolist())

['CustomerID', 'CompanyName', 'ContactName', 'ContactTitle', 'Address', 'City', 'Region', 'PostalCode', 'Country', 'Phone', 'Fax', 'state']


Loops

In [6]:
for index, row in department_df.iterrows():

    try:
        query = f"INSERT INTO Department VALUES({row['dept_id']}, '{row['dept_name']}', {row['dept_head_id']})"
        source_connection.execute(query)

    except pyodbc.Error:
        print(query)

    source_connection.commit()

for index, row in bonus_df.iterrows():

    try:
        query = f"INSERT INTO Bonus VALUES({row['emp_id']}, '{row['bonus_date']}', {row['bonus_amount']})"
        source_connection.execute(query)

    except pyodbc.Error:
        print(query)

    source_connection.commit()

for index, row in state_df.iterrows():

    try:
        query = f"INSERT INTO State VALUES('{row['state_id']}', '{row['state_name']}', '{row['state_capital'].replace("'", "''")}', '{row['country']}', '{row['region']}')"
        source_connection.execute(query)

    except pyodbc.Error:
        print(query)

    source_connection.commit()

for index, row in employee_df.iterrows():

    try:
        query = f"INSERT INTO Employee VALUES({row['emp_id']}, {row['manager_id']}, '{row['emp_fname']}', '{row['emp_lname']}', {row['dept_id']}, '{row['street']}', '{row['city']}', '{row['state']}', {row['zip_code']}, {row['phone']}, '{row['status']}', {row['ss_number']}, {row['salary']}, '{row['start_date']}', '{row['birth_date']}', '{row['bene_health_ins']}', '{row['bene_life_ins']}', '{row['bene_day_care']}', '{row['sex']}')"
        source_connection.execute(query)

    except pyodbc.Error:
        print(query)

    source_connection.commit()

for index, row in shippers_df.iterrows():

    try:
        query = f"INSERT INTO Shippers VALUES({row['ShipperID']}, '{row['CompanyName']}', {row['Phone']})"
        source_connection.execute(query)

    except pyodbc.Error:
        print(query)

    source_connection.commit()

for index, row in customer_merge_df.iterrows():

    try:
        query = f"INSERT INTO Customer VALUES('{row['CustomerID']}', '{row['CompanyName']}', '{row['ContactName']}', '{row['ContactTitle']}', '{row['Address']}', '{row['City']}', '{row['Region']}', '{row['PostalCode']}', '{row['Country']}', '{row['Phone']}', '{row['Fax']}', '{row['state']}')"
        source_connection.execute(query)

    except pyodbc.Error:
        print(query)

    source_connection.commit()

for index, row in orders_df.iterrows():

    try:
        query = f"INSERT INTO Orders VALUES({row['OrderID']}, '{row['CustomerID']}', {row['EmployeeID']}, '{row['OrderDate']}', '{row['RequiredDate']}', '{row['ShippedDate']}', '{row['ShipVia']}', '{row['Freight']}', '{row['ShipName'].replace("'", "''")}', '{row['ShipAddress'].replace("'", "''")}', '{row['ShipCity']}', '{row['ShipRegion']}', '{row['ShipPostalCode']}', '{row['ShipCountry']}')"
        source_connection.execute(query)

    except pyodbc.Error:
        print(query)

    source_connection.commit()

for index, row in order_details_df.iterrows():

    try:
        query = f"INSERT INTO Order_Details VALUES({row['OrderID']}, {row['ProductID']}, {row['UnitPrice']}, {row['Quantity']}, {row['Discount']})"
        source_connection.execute(query)

    except pyodbc.Error:
        print(query)

    source_connection.commit()

for index, row in sales_order_df.iterrows():

    try:
        query = f"INSERT INTO Sales_Order VALUES({row['id']}, {row['cust_id']}, '{row['order_date']}', '{row['region']}', '{row['sales_rep']}')"
        source_connection.execute(query)

    except pyodbc.Error:
        print(query)

    source_connection.commit()
    
for index, row in sales_order_item_df.iterrows():

    try:
        query = f"INSERT INTO Sales_Order_Item VALUES({row['id']}, {row['line_id']}, {row['prod_id']}, {row['quantity']}, '{row['ship_date']}')"
        source_connection.execute(query)

    except pyodbc.Error:
        print(query)

    source_connection.commit()

for index, row in categories_df.iterrows():

    try:
        query = f"INSERT INTO Categories VALUES({row['CategoryID']}, '{row['CategoryName']}', '{row['Description']}', '{row['Picture']}')"
        source_connection.execute(query)

    except pyodbc.Error:
        print(query)

    source_connection.commit()

for index, row in products_df.iterrows():

    try:
        query = f"INSERT INTO Products VALUES({row['ProductID']}, '{row['ProductName']}', {row['SupplierID']}, {row['CategoryID']}, '{row['QuantityPerUnit']}', {row['UnitPrice']}, {row['UnitsInStock']}, {row['UnitsOnOrder']}, {row['ReorderLevel']}, '{row['Discontinued']}')"
        source_connection.execute(query)

    except pyodbc.Error:
        print(query)

    source_connection.commit()

for index, row in product_df.iterrows():

    try:
        query = f"INSERT INTO Product VALUES({row['id']}, '{row['name']}', '{row['description']}', '{row['prod_size']}', '{row['color']}', {row['quantity']}, {row['unit_price']}, '{row['picture_name']}', '{row['Category']}')"
        source_connection.execute(query)

    except pyodbc.Error:
        print(query)

    source_connection.commit()

for index, row in territories_df.iterrows():

    try:
        query = f"INSERT INTO Territories VALUES({row['TerritoryID']}, '{row['TerritoryDescription']}', {row['RegionID']})"
        source_connection.execute(query)

    except pyodbc.Error:
        print(query)

    source_connection.commit()

for index, row in employee_territories_df.iterrows():

    try:
        query = f"INSERT INTO EmployeeTerritories VALUES({row['EmployeeID']}, {row['TerritoryID']})"
        source_connection.execute(query)

    except pyodbc.Error:
        print(query)

    source_connection.commit()

for index, row in employees_df.iterrows():

    try:
        query = f"INSERT INTO Employees VALUES({row['EmployeeID']}, '{row['LastName']}', '{row['FirstName']}', '{row['Title']}', '{row['TitleOfCourtesy']}', {row['BirthDate']}, {row['HireDate']}, '{row['Address']}', '{row['City']}', '{row['Region']}', {row['PostalCode']}, '{row['Country']}', '{row['HomePhone']}', '{row['Extension']}', '{row['Photo']}', '{row['Notes']}', '{row['ReportsTo']}', '{row['PhotoPath']}')"
        source_connection.execute(query)

    except pyodbc.Error:
        print(query)

    source_connection.commit()

for index, row in region_df.iterrows():

    try:
        query = f"INSERT INTO Region VALUES({row['RegionID']}, '{row['RegionDescription']}')"
        source_connection.execute(query)

    except pyodbc.Error:
        print(query)

    source_connection.commit()

for index, row in suppliers_df.iterrows():

    try:
        query = f"INSERT INTO Suppliers VALUES({row['SupplierID']}, '{row['CompanyName']}', '{row['ContactName']}', '{row['ContactTitle']}', '{row['Address']}', '{row['City']}', '{row['Region']}', {row['PostalCode']}, '{row['Country']}', '{row['Phone']}', '{row['Fax']}', '{row['HomePage']}')"
        source_connection.execute(query)

    except pyodbc.Error:
        print(query)

    source_connection.commit()

INSERT INTO Employee VALUES(1013, 703, 'Joseph', 'Barker', 500, '58 West Drive', 'Bedford', 'MA', 1730, 6175558021, 'A', 23470756, 27290.0, '10-Sep-2019', '14-Feb-1996', 'N', 'Y', 'N', 'M')
INSERT INTO Employee VALUES(102, 501, 'Fran', 'Whitney', 100, '49 East Washington Street', 'Needham', 'MA', 2192, 6175553985, 'A', 17349033, 45700.0, '26-Feb-2013', '05-Jun-1985', 'Y', 'Y', 'N', 'F')
INSERT INTO Employee VALUES(1021, 902, 'Paul', 'Sterling', 200, '112 Endicott Street', 'Concord', 'MA', 1742, 5085550295, 'A', 37846595, 64900.0, '28-Oct-2019', '27-Feb-1977', 'Y', 'Y', 'Y', 'M')
INSERT INTO Employee VALUES(1039, 902, 'Shih Lin', 'Chao', 200, '59 Holyoke Street', 'Lexington', 'MA', 2173, 6175555921, 'A', 46973741, 33890.0, '11-Nov-2019', '12-Dec-1996', 'N', 'Y', 'Y', 'M')
INSERT INTO Employee VALUES(105, 501, 'Matthew', 'Cobb', 100, '77 Pleasant Street', 'Waltham', 'MA', 2154, 6175553840, 'A', 52345739, 62000.0, '02-Jul-2013', '04-Dec-1987', 'Y', 'Y', 'N', 'M')
INSERT INTO Employee VALU

In [7]:
# Suppliers heeft geen create table?
# Region heeft geen create table?
# Create Table of Loop van Territory/Territories matched niet.
# Product/Products kloppen niet met de input data.

# Employee moet gemerged worden

# EmployeeTerritories foreign key constraint met Employee
# Sales_Order foreign key constraint met Customer
# Sales_Order_Item foreign key constraint met Sales_Order
# Order_Details foreign key constraint met Orders
# Orders foreign key constraint met Customer